## GEOSeMOSYS - exploring linear programming geospatial energy modelling 

Nandi Moksnes, Mark Howells............

This Jupyter Notebook is dectription of how to reproduce the results in the paper:
For any questions please contact Nandi Moksnes: nandi@kth.se

## 1. The files in "GIS_data" are downloaded and placed in a "temp" folder.

In [6]:
from Download_files.py import *

download_url_data(input_data/GIS_URL.txt temp)
unzip_all(input_data/GIS_URL.txt)

## 2. The files are then projected and clipped to the administrative boundaries.

In [1]:
from Project_GIS import *
merge_minigrid('..\Projected_files')

#main('../GIS_data')

## 3. Through QGIS make raster to point layer and save

1) masked_UMT37S_ken_ppp_2015_UNadj: SAGA raster to point 

2) This step is



## 4. The GIS layers are prepared to for a heuristic approximation for electrified settlements

In [ ]:
from settlement_build import *


### Import gridded file from Zenodo for Kenya

In [6]:
from Download_files import *

download_url_data('input_data/zenodo.txt', '../Projected_files')



()

## 4. One central part of the geospatial energy modelling dimension is assessing the energy resources at the location. This is done through the capacity factors from renewable.ninja for wind and solar PV for each of the 376 cells downloaded from zenodo.

In [ ]:
import time
import subprocess
token = 'ed519952eff7850cece8c746347fee2d068ab988'
type = "solar"

j = 0
max_hour =3601
start_hour = time.time()
while True:
    ### Do other stuff, it won't be blocked
    time.sleep(0.1)
    print("looping...")

    ### This will be updated every loop
    remaining_hour = max_hour + start_hour - time.time()
    print("%s seconds remaining" % int(remaining_hour))

    ### Countdown finished, ending loop
    if remaining_hour <= 0:
        break


    count = 0
    max = 60
    start = time.time()
    while True:
        token_ = "ed519952eff7850cece8c746347fee2d068ab988"
        type = "wind"
        ### Do other stuff, it won't be blocked
        time.sleep(0.1)
        print("looping...")

        ### This will be updated every loop
        remaining = max + start - time.time()
        print("%s seconds remaining" % int(remaining))

        ### Countdown finished, ending loop
        if remaining <= 0:
            break

        while count <=6:
            csvfiles = r'.\temp\solar1-6.csv'
            csvfilesout = r'.\temp\solar1-6_out.csv'
            subprocess.call("RScript GEOSeMOSYS_download.r"+" "+ token_+" "+type+" "+ csvfiles+" "+csvfilesout, shell=True)
            count +=1
    j+=1

## 4. The GIS files needs to be scaled down to a gridded dataset for populating the GIS attributes to estimate the electrified settelments.

In [ ]:
#This code is modified from https://github.com/babakkhavari/OnSSET-GIS-Extraction
#This section creates raster to point, and assigns the different transmission lines and nightime lights to the same geopandas df


import os
import sys
import geopandas as gpd
import os
import fiona
import ipywidgets as widgets
from IPython.display import display
from rasterstats import zonal_stats
import rasterio
import rasterio.fill
from shapely.geometry import shape, mapping
import json
import numpy as np
import tkinter as tk
from tkinter import filedialog, messagebox
import gdal
import datetime
import warnings
import pandas as pd
import scipy.spatial
from osgeo import gdal,ogr
warnings.filterwarnings('ignore')

root = tk.Tk()
root.withdraw()
root.attributes("-topmost", True)
def settlement_process():
    open()

    return()

#måste lösas först
def raster_to_numpyarray():  # make raster to point
    pop_file = os.path.join(Projected_files_path+r'\masked_UMT37S_ken_ppp_2015_UNadj.TIF')
    pop_shp = os.path.join(Projected_files_path+r'\pop.shp')

    def pixelOffset2coord(raster, xOffset, yOffset):
        geotransform = raster.GetGeoTransform()
        originX = geotransform[0]
        originY = geotransform[3]
        pixelWidth = geotransform[1]
        pixelHeight = geotransform[5]
        coordX = originX + pixelWidth * xOffset
        coordY = originY + pixelHeight * yOffset
        return coordX, coordY

    def raster2array(rasterfn):
        raster = gdal.Open(rasterfn)
        band = raster.GetRasterBand(1)
        array = band.ReadAsArray()
        return array

    def array2shp(array, outSHPfn, rasterfn):

        # max distance between points
        raster = gdal.Open(rasterfn)
        geotransform = raster.GetGeoTransform()
        pixelWidth = geotransform[1]

        # wkbPoint
        shpDriver = ogr.GetDriverByName("ESRI Shapefile")
        if os.path.exists(outSHPfn):
            shpDriver.DeleteDataSource(outSHPfn)
        outDataSource = shpDriver.CreateDataSource(outSHPfn)
        outLayer = outDataSource.CreateLayer(outSHPfn, geom_type=ogr.wkbPoint)
        featureDefn = outLayer.GetLayerDefn()
        outLayer.CreateField(ogr.FieldDefn("ID", ogr.OFTInteger))

        # array2dict
        point = ogr.Geometry(ogr.wkbPoint)
        row_count = array.shape[0]
        for ridx, row in enumerate(array):
            if ridx % 100 == 0:
                print("{0} of {1} rows processed".format(ridx, row_count))
            for cidx, value in enumerate(row):
                Xcoord, Ycoord = pixelOffset2coord(raster, cidx, ridx)
                point.AddPoint(Xcoord, Ycoord)
                outFeature = ogr.Feature(featureDefn)
                outFeature.SetGeometry(point)
                outLayer.CreateFeature(outFeature)
                outFeature.SetField("ID", outFeature.GetFID())
                outLayer.SetFeature(outFeature)
                outFeature.Destroy()
        outDataSource.Destroy()

    array = raster2array(pop_file)
    array2shp(array, pop_shp, pop_file)



# messagebox.showinfo('OnSSET', 'Select the clusters')
# clusters = gpd.read_file(filedialog.askopenfilename(filetypes=(("shapefile", "*.shp"), ("all files", "*.*"))))
#
# ##widget ta bort - peka mot Population
# popunit = widgets.Dropdown(options=clusters.head(),
#                            value=None,
#                            description='Population:',
#                            disabled=False)
# display(popunit)
# x = popunit.value



#Geopandas code Babak
# #Processing Rasters
#for nighttime light
def processing_raster(name, method, clusters):
    messagebox.showinfo('OnSSET', 'Select the ' + name + ' map')
    raster = rasterio.open(filedialog.askopenfilename(filetypes=(("rasters", "*.tif"), ("all files", "*.*"))))

    clusters = zonal_stats(
        clusters,
        raster.name,
        stats=[method],
        prefix=name, geojson_out=True, all_touched=True)

    print(datetime.datetime.now())
    return clusters





#Finalizing rasters
#Zonal statistics geopandas output is not a dataframe
def finalizing_rasters(workspace, clusters, crs):
    output = workspace + r'\placeholder.geojson'
    with open(output, "w") as dst:
        collection = {
            "type": "FeatureCollection",
            "features": list(clusters)}
        dst.write(json.dumps(collection))

    clusters = gpd.read_file(output)
    os.remove(output)

    print(datetime.datetime.now())
    return clusters

#Preparing for vectors
def preparing_for_vectors(workspace, clusters, crs):
    clusters.crs = {'init' :'epsg:4326'}
    clusters = clusters.to_crs({ 'init': crs})
    points = clusters.copy()
    points["geometry"] = points["geometry"].centroid
    points.to_file(workspace + r'\clusters_cp.shp', driver='ESRI Shapefile')
    print(datetime.datetime.now())
    return clusters

#Processing Lines
#Geopandas kan inte hantera lines så det blir points
def processing_lines(name, admin, crs, workspace, clusters):
    messagebox.showinfo('OnSSET', 'Select the ' + name + ' map')
    lines = gpd.read_file(filedialog.askopenfilename(filetypes=(("shapefile", "*.shp"), ("all files", "*.*"))))

    lines_clip = clip.clip_shp(lines, admin)
    lines_clip.crs = {'init': 'epsg:4326'}
    lines_proj = lines_clip.to_crs({'init': crs})

    lines_proj.to_file(workspace + r"\ " + name + "_proj.shp", driver='ESRI Shapefile')

    line = fiona.open(workspace + r"\ " + name + "_proj.shp")
    firstline = line.next()

    schema = {'geometry': 'Point', 'properties': {'id': 'int'}, }
    with fiona.open(workspace + r"\ " + name + "_proj_points.shp", "w", "ESRI Shapefile", schema) as output:
        for lines in line:
            if lines["geometry"] is not None:
                first = shape(lines['geometry'])
                length = first.length
                for distance in range(0, int(length), 100):
                    point = first.interpolate(distance)
                    output.write({'geometry': mapping(point), 'properties': {'id': 1}})

    lines_f = fiona.open(workspace + r"\ " + name + "_proj_points.shp")
    lines = gpd.read_file(workspace + r"\ " + name + "_proj.shp")
    points = fiona.open(workspace + r'\clusters_cp.shp')

    ##
    geoms1 = [shape(feat["geometry"]) for feat in lines_f]
    s1 = [np.array((geom.xy[0][0], geom.xy[1][0])) for geom in geoms1]
    s1_arr = np.array(s1)

    geoms2 = [shape(feat["geometry"]) for feat in points]
    s2 = [np.array((geom.xy[0][0], geom.xy[1][0])) for geom in geoms2]
    s2_arr = np.array(s2)

    def do_kdtree(combined_x_y_arrays, points):
        mytree = scipy.spatial.cKDTree(combined_x_y_arrays)
        dist, indexes = mytree.query(points)
        return dist, indexes

    ##ger alla vektorer som overlappar med linje. TA BORT!!!!
    def vector_overlap(vec, settlementfile, column_name):
        vec.drop(vec.columns.difference(["geometry"]), 1, inplace=True)
        a = gpd.sjoin(settlementfile, vec, op='intersects')
        a[column_name + '2'] = 0
        return a

    results1, results2 = do_kdtree(s1_arr, s2_arr)

    z = results1.tolist()
    clusters[name + 'Dist'] = z
    clusters[name + 'Dist'] = clusters[name + 'Dist'] / 1000

    ##Ta bort
    a = vector_overlap(lines, clusters, name + 'Dist')

    ##ta bort
    clusters = pd.merge(left=clusters, right=a[['id', name + 'Dist2']], on='id', how='left')
    clusters.drop_duplicates(subset="id", keep="first", inplace=True)

    ## ta bort
    clusters.loc[clusters[name + 'Dist2'] == 0, name + 'Dist'] = 0

    # ta bort
    del clusters[name + 'Dist2']
    print(datetime.datetime.now())
    return clusters

#Processing points
def processing_points(name, admin, crs, workspace, clusters):
    messagebox.showinfo('OnSSET', 'Select the ' + name + ' map')
    points = gpd.read_file(filedialog.askopenfilename(filetypes=(("shapefile", "*.shp"), ("all files", "*.*"))))

    points_clip = clip.clip_shp(points, admin)
    points_clip.crs = {'init': 'epsg:4326'}
    points_proj = points_clip.to_crs({'init': crs})

    points_proj.to_file(workspace + r"\ " + name + "_proj.shp", driver='ESRI Shapefile')

    points_f = fiona.open(workspace + r"\ " + name + "_proj.shp")
    points = gpd.read_file(workspace + r"\ " + name + "_proj.shp")
    points2 = fiona.open(workspace + r'\clusters_cp.shp')

    geoms1 = [shape(feat["geometry"]) for feat in points_f]
    s1 = [np.array((geom.xy[0][0], geom.xy[1][0])) for geom in geoms1]
    s1_arr = np.array(s1)

    geoms2 = [shape(feat["geometry"]) for feat in points2]
    s2 = [np.array((geom.xy[0][0], geom.xy[1][0])) for geom in geoms2]
    s2_arr = np.array(s2)

    def do_kdtree(combined_x_y_arrays, points):
        mytree = scipy.spatial.cKDTree(combined_x_y_arrays)
        dist, indexes = mytree.query(points)
        return dist, indexes

    ##Ta bort
    def vector_overlap(vec, settlementfile, column_name):
        vec.drop(vec.columns.difference(["geometry"]), 1, inplace=True)
        a = gpd.sjoin(settlementfile, vec, op='intersects')
        a[column_name + '2'] = 0
        return a

    results1, results2 = do_kdtree(s1_arr, s2_arr)

    z = results1.tolist()
    clusters[name + 'Dist'] = z
    clusters[name + 'Dist'] = clusters[name + 'Dist'] / 1000

    # ta bort
    a = vector_overlap(points, clusters, name + 'Dist')

    # ta bort
    clusters = pd.merge(left=clusters, right=a[['id', name + 'Dist2']], on='id', how='left')
    clusters.drop_duplicates(subset="id", keep="first", inplace=True)
    # ta bort
    clusters.loc[clusters[name + 'Dist2'] == 0, name + 'Dist'] = 0
    # ta bort
    del clusters[name + 'Dist2']
    print(datetime.datetime.now())
    return clusters

#Conditioning
def conditioning(clusters, workspace, popunit):
    clusters = clusters.to_crs({'init': 'epsg:4326'})

    clusters = clusters.rename(columns={"NightLight": "NightLights", popunit: "Pop", "GridCellAr": "GridCellArea"})

    if "landcovermajority" in clusters:
        clusters = clusters.rename(columns={"landcovermajority": "LandCover"})

    if "elevationmean" in clusters:
        clusters = clusters.rename(columns={"elevationmean": "Elevation"})

    if "sl_majority" in clusters:
        clusters = clusters.rename(columns={"sl_majority": "Slope"})

    if "ghimean" in clusters:
        clusters = clusters.rename(columns={"ghimean": "GHI"})

    if "traveltimemean" in clusters:
        clusters["traveltimemean"] = clusters["traveltimemean"] / 60
        clusters = clusters.rename(columns={"traveltimemean": "TravelHours"})
    elif "TravelHour" in clusters:
        clusters = clusters.rename(columns={"TravelHour": "TravelHours"})

    if "windmean" in clusters:
        clusters = clusters.rename(columns={"windmean": "WindVel"})

    if "Residentia" in clusters:
        clusters = clusters.rename(columns={"Resudentia": "ResidentialDemandTierCustom"})
    elif "customdemandmean" in clusters:
        clusters = clusters.rename(columns={"customdemandmean": "ResidentialDemandTierCustom"})
    else:
        clusters["ResidentialDemandTierCustom"] = 0

    if "Substation" in clusters:
        clusers = clusters.rename(columns={"Substation": "SubstationDist"})
    elif "SubstationDist" not in clusters:
        clusters["SubstationDist"] = 99

    if "CurrentHVL" in clusters:
        clusters = clusters.rename(columns={"CurrentHVL": "Existing_HVDist"})

    if "CurrentMVL" in clusters:
        clusters = clusters.rename(columns={"CurrentMVL": "Existing_MVDist"})

    if "PlannedHVL" in clusters:
        clusters = clusters.rename(columns={"PlannedHVL": "Planned_HVDist"})

    if "PlannedMVL" in clusters:
        clusters = clusters.rename(columns={"PlannedMVL": "Planned_MVDist"})

    if "Existing_HVDist" in clusters:
        clusters = clusters.rename(columns={"Existing_HVDist": "CurrentHVLineDist"})
        if "Planned_HVDist" in clusters:
            mask = (clusters['Planned_HVDist'] > clusters['CurrentHVLineDist'])
            clusters['Planned_HVDist'][mask] = clusters['CurrentHVLineDist']
            clusters = clusters.rename(columns={"Planned_HVDist": "PlannedHVLineDist"})
        else:
            clusters["PlannedHVLineDist"] = clusters["CurrentHVLineDist"]
    elif "Existing_HVDist" not in clusters and "Planned_HVDist" not in clusters:
        clusters["PlannedHVLineDist"] = 99
        clusters["CurrentHVLineDist"] = 0
    else:
        clusters["CurrentHVLineDist"] = 0
        clusters = clusters.rename(columns={"Planned_HVDist": "PlannedHVLineDist"})

    if "Existing_MVDist" in clusters:
        clusters = clusters.rename(columns={"Existing_MVDist": "CurrentMVLineDist"})
        if "Planned_MVDist" in clusters:
            mask = (clusters['Planned_MVDist'] > clusters['CurrentMVLineDist'])
            clusters['Planned_MVDist'][mask] = clusters['CurrentMVLineDist']
            clusters = clusters.rename(columns={"Planned_MVDist": "PlannedMVLineDist"})
        else:
            clusters["PlannedMVLineDist"] = clusters["CurrentMVLineDist"]
    elif "Existing_MVDist" not in clusters and "Planned_MVDist" not in clusters:
        clusters["PlannedMVLineDist"] = 99
        clusters["CurrentMVLineDist"] = 0
    else:
        clusters["CurrentMVLineDist"] = 0
        clusters = clusters.rename(columns={"Planned_MVDist": "PlannedMVLineDist"})

    if "RoadDist" not in clusters:
        clusters["RoadDist"] = 0

    if "Transforme" in clusters:
        clusters = clusters.rename(columns={"Transforme": "TransformerDist"})
    elif "TransformerDist" not in clusters:
        clusters["TransformerDist"] = 0

    if "Hydropower" not in clusters:
        clusters["Hydropower"] = 0

    if "Hydropow_1" in clusters:
        clusters = clusters.rename(columns={"Hydropow_1": "HydropowerDist"})
    elif 'HydropowerDist' not in clusters:
        clusters["HydropowerDist"] = 99

    if "Hydropow_2" in clusters:
        clusters = clusters.rename(columns={"Hydropow_2": "HydropowerFID"})
    elif "HydropowerFID" not in clusters:
        clusters["HydropowerFID"] = 0

    if "IsUrban" not in clusters:
        clusters["IsUrban"] = 0

    if "PerCapitaD" not in clusters:
        clusters["PerCapitaDemand"] = 0
    else:
        clusters = clusters.rename(columns={"PerCapitaD": "PerCapitaDemand"})

    if "HealthDema" not in clusters:
        clusters["HealthDemand"] = 0
    else:
        clusters = clusters.rename(columns={"HealthDema": "HealthDemand"})

    if "EducationD" not in clusters:
        clusters["EducationDemand"] = 0
    else:
        clusters = clusters.rename(columns={"EducationD": "EducationDemand"})

    if "AgriDemand" not in clusters:
        clusters["AgriDemand"] = 0

    if "Commercial" not in clusters:
        clusters["CommercialDemand"] = 0
    else:
        clusters = clusters.rename(columns={"Commercial": "CommercialDemand"})

    if "Conflict" not in clusters:
        clusters["Conflict"] = 0

    if "Electrific" not in clusters:
        clusters["ElectrificationOrder"] = 0
    else:
        clusters = clusters.rename(columns={"Electrific": "ElectrificationOrder"})

    if "Resident_1" not in clusters:
        clusters["ResidentialDemandTier1"] = 7.74
    else:
        clusters = clusters.rename(columns={"Resident_1": "ResidentialDemandTier1"})

    if "Resident_2" not in clusters:
        clusters["ResidentialDemandTier2"] = 43.8
    else:
        clusters = clusters.rename(columns={"Resident_2": "ResidentialDemandTier2"})

    if "Resident_3" not in clusters:
        clusters["ResidentialDemandTier3"] = 160.6
    else:
        clusters = clusters.rename(columns={"Resident_3": "ResidentialDemandTier3"})

    if "Resident_4" not in clusters:
        clusters["ResidentialDemandTier4"] = 423.4
    else:
        clusters = clusters.rename(columns={"Resident_4": "ResidentialDemandTier4"})

    if "Resident_5" not in clusters:
        clusters["ResidentialDemandTier5"] = 598.6
    else:
        clusters = clusters.rename(columns={"Resident_5": "ResidentialDemandTier5"})

    clusters["X_deg"] = clusters.geometry.centroid.x

    clusters["Y_deg"] = clusters.geometry.centroid.y

    clusters.to_file(workspace + r"\output.shp", driver='ESRI Shapefile')
    clusters.to_file(workspace + r"\output.csv", driver='CSV')
    print(datetime.datetime.now())
    return clusters



def merge_transmission(proj_path):
    current = os.getcwd()
    os.chdir(proj_path)
    files = os.listdir(proj_path)
    shapefiles = []
    for file in files:
        if file.endswith('.shp'):
            f = os.path.abspath(file)
            shapefiles += [f]
    keyword = 'kV'
    tmiss_list = []
    for fname in shapefiles:
        if keyword in fname:
            shpfile = gpd.read_file(fname)
            tmiss_list += [shpfile]

    gdf = pd.concat([shp for shp in tmiss_list]).pipe(gpd.GeoDataFrame)
    gdf.to_file("../Projected_files/Concat_Transmission_lines_UMT37S.shp")
    os.chdir(current)

if __name__ == "__main__":
    Projected_files_path = sys.argv[1]
    #merge_transmission(Projected_files_path)
    raster_to_numpyarray()
    crs = 'EPSG:3395'

## 5. The electrification start forms the demand centers which are classified as electrified and un-electrified within the gridded cells of the 376 cells of Kenya

In [ ]:
#
# Author: KTH dESA Last modified by Nandi Moksnes
# Date: 2020-06
# Python version: 3.7
import os
import sys
from ..src.prep_elecstart import settlement_process
from onsset import *
import pandas as pd

#Inputdata that you need to define
##Snakemake
def main(GIS_files, specs_path):
    specs = pd.read_excel(specs_path, index_col=0)
    ##Snakemake
    output_dir = '../output/'
    settlements_in = settlement_process()
    settlements_out = os.path.join(output_dir,'settlements.csv')
    country = "Kenya"
    settlements_out_csv = output_dir + '.csv' # os.path.join(output_dir, '{}.csv'.format(country))

    onsseter = SettlementProcessor(settlements_in)

    pop_actual = specs.loc["Kenya", SPE_POP]
    pop_future = specs.loc["Kenya", SPE_POP_FUTURE]
    urban_current = specs.loc["Kenya", SPE_URBAN]
    urban_future = specs.loc["Kenya", SPE_URBAN_FUTURE]
    urban_cutoff = specs.loc["Kenya", SPE_URBAN_CUTOFF]
    start_year = int(specs.loc["Kenya", SPE_START_YEAR])
    end_year = int(specs.loc["Kenya", SPE_END_YEAR])

    elec_actual = specs.loc[country, SPE_ELEC]
    pop_cutoff = specs.loc[country, SPE_POP_CUTOFF1]
    min_night_lights = specs.loc[country, SPE_MIN_NIGHT_LIGHTS]
    max_grid_dist = specs.loc[country, SPE_MAX_GRID_DIST]
    max_road_dist = specs.loc[country, SPE_MAX_ROAD_DIST]
    pop_tot = specs.loc[country, SPE_POP]
    pop_cutoff2 = specs.loc[country, SPE_POP_CUTOFF2]
    dist_to_trans = specs.loc[country, SPE_DIST_TO_TRANS]

    urban_cutoff, urban_modelled = onsseter.calibrate_pop_and_urban(pop_actual, pop_future, urban_current,
                                                                    urban_future, urban_cutoff, start_year, end_year)

    min_night_lights, dist_to_trans, max_grid_dist, max_road_dist, elec_modelled, pop_cutoff, pop_cutoff2, rural_elec_ratio, urban_elec_ratio = \
        onsseter.elec_current_and_future(elec_actual, pop_cutoff, dist_to_trans, min_night_lights, max_grid_dist,
                                         max_road_dist, pop_tot, pop_cutoff2, start_year)

    onsseter.grid_reach_estimate(start_year, gridspeed=9999)

    specs.loc[country, SPE_URBAN_MODELLED] = urban_modelled
    specs.loc[country, SPE_URBAN_CUTOFF] = urban_cutoff
    specs.loc[country, SPE_MIN_NIGHT_LIGHTS] = min_night_lights
    specs.loc[country, SPE_MAX_GRID_DIST] = max_grid_dist
    specs.loc[country, SPE_MAX_ROAD_DIST] = max_road_dist
    specs.loc[country, SPE_ELEC_MODELLED] = elec_modelled
    specs.loc[country, SPE_POP_CUTOFF1] = pop_cutoff
    specs.loc[country, SPE_POP_CUTOFF2] = pop_cutoff2
    specs.loc[country, 'rural_elec_ratio'] = rural_elec_ratio
    specs.loc[country, 'urban_elec_ratio'] = urban_elec_ratio


    try:
        specs.to_excel(specs_path)
    except ValueError:
        specs.to_excel(specs_path + '.xlsx')

    onsseter.df.to_csv(settlements_out_csv, index=False)

def elec_current_and_future(self, elec_actual, pop_cutoff, dist_to_trans, min_night_lights,
                            max_grid_dist, max_road_dist, pop_tot, pop_cutoff2, start_year):
    """
    Calibrate the current electrification status, and future 'pre-electrification' status
    """
    urban_pop = (self.df.loc[self.df[SET_URBAN] == 2, SET_POP_CALIB].sum())  # Calibrate current electrification
    rural_pop = (self.df.loc[self.df[SET_URBAN] < 2, SET_POP_CALIB].sum())  # Calibrate current electrification
    total_pop = self.df[SET_POP_CALIB].sum()
    total_elec_ratio = elec_actual
    urban_elec_ratio = 0.98
    rural_elec_ratio = 0.393
    factor = (total_pop * total_elec_ratio) / (urban_pop * urban_elec_ratio + rural_pop * rural_elec_ratio)
    urban_elec_ratio *= factor
    rural_elec_ratio *= factor
    # print('factor: ' + str(factor))

    logging.info('Calibrate current electrification')
    is_round_two = False
    grid_cutoff2 = 5
    road_cutoff2 = 5
    count = 0
    prev_vals = []
    accuracy = 0.01
    max_iterations_one = 30
    max_iterations_two = 60
    # self.df[SET_ELEC_CURRENT] = 0

    # if max(self.df['TransformerDist']) > 0:
    #    self.df['GridDistCalibElec'] = self.df['TransformerDist']
    #    priority = 1
    # elif max(self.df['CurrentMVLineDist']) > 0:
    #    self.df['GridDistCalibElec'] = self.df['CurrentMVLineDist']
    #    priority = 1
    # else:
    #    self.df['GridDistCalibElec'] = self.df['CurrentHVLineDist']
    #    priority = 2
    #
    #
    # condition = 0
    # while condition == 0:
    #     # Assign the 1 (electrified)/0 (un-electrified) values to each cell
    #     # urban_electrified = 0.159853988426699 * 17573607 * 0.487
    #     urban_electrified = urban_pop * urban_elec_ratio
    #     # urban_electrified = urban_electrified_modelled * self.df[SET_POP_CALIB].sum() * urban_elec_access
    #     # rural_electrified = (1 - 0.159853988426699) * 17573607 * 0.039
    #     rural_electrified = rural_pop * rural_elec_ratio
    #     # rural_electrified = (1 - urban_electrified_modelled) * self.df[SET_POP_CALIB].sum() * rural_elec_access
    #     if priority == 1:
    #         self.df.loc[(self.df['GridDistCalibElec'] < 15) & (self.df[SET_NIGHT_LIGHTS] > 0) & (self.df[SET_POP_CALIB] > 5), SET_ELEC_CURRENT] = 1
    #         # self.df.loc[(self.df[SET_NIGHT_LIGHTS] > 0) & (self.df[SET_POP_CALIB] > 50), SET_ELEC_CURRENT] = 1
    #         # self.df.loc[(self.df['GridDistCalibElec'] < 0.8), SET_ELEC_CURRENT] = 1
    #         urban_elec_ratio = urban_electrified / (self.df.loc[(self.df[SET_ELEC_CURRENT] == 1) & (self.df[SET_URBAN] == 2), SET_POP_CALIB].sum())
    #         rural_elec_ratio = rural_electrified / (self.df.loc[(self.df[SET_ELEC_CURRENT] == 1) & (self.df[SET_URBAN] < 2), SET_POP_CALIB].sum())
    #         pop_elec = self.df.loc[self.df[SET_ELEC_CURRENT] == 1, SET_POP_CALIB].sum()
    #         elec_modelled = pop_elec / pop_tot
    #     else:
    #         self.df.loc[(self.df[SET_NIGHT_LIGHTS] > 0) & (self.df['GridDistCalibElec'] < 5) |  | (self.df[SET_POP_CALIB] > 7000) | (self.df[SET_ROAD_DIST]<3), SET_ELEC_CURRENT] = 1
    #         # self.df.loc[(self.df['GridDistCalibElec'] < 0.8), SET_ELEC_CURRENT] = 1
    #         urban_elec_ratio = (self.df.loc[(self.df[SET_ELEC_CURRENT] == 1) & (
    #                     self.df[SET_URBAN] == 2), SET_POP_CALIB].sum()) / urban_electrified
    #         rural_elec_ratio = (self.df.loc[(self.df[SET_ELEC_CURRENT] == 1) & (
    #                     self.df[SET_URBAN] < 2), SET_POP_CALIB].sum()) / rural_electrified
    #         pop_elec = self.df.loc[self.df[SET_ELEC_CURRENT] == 1, SET_POP_CALIB].sum()
    #         elec_modelled = pop_elec / pop_tot

    while True:
        self.df[SET_ELEC_CURRENT] = self.df.apply(lambda row:
                                                  1
                                                  if ((row[SET_NIGHT_LIGHTS] > 0 or
                                                       row[SET_POP_CALIB] > pop_cutoff and
                                                       row['CurrentHVLineDist'] < max_grid_dist or
                                                       row[SET_ROAD_DIST] < max_road_dist))
                                                  # or (row[SET_POP_CALIB] > pop_cutoff2 and
                                                  # (row['CurrentHVLineDist'] < grid_cutoff2 or
                                                  # row[SET_ROAD_DIST] < road_cutoff2))
                                                  else 0, axis=1)

        # Get the calculated electrified ratio, and limit it to within reasonable boundaries
        pop_elec = self.df.loc[self.df[SET_ELEC_CURRENT] == 1, SET_POP_CALIB].sum()
        elec_modelled = pop_elec / pop_tot

        if elec_modelled == 0:
            elec_modelled = 0.01
        elif elec_modelled == 1:
            elec_modelled = 0.99

        if abs(elec_modelled - elec_actual) < accuracy:
            break
        elif not is_round_two:
            min_night_lights = \
            sorted([0, min_night_lights - min_night_lights * 2 * (elec_actual - elec_modelled) / elec_actual, 1])[1]
            pop_cutoff = \
            sorted([0.01, pop_cutoff - pop_cutoff * 0.5 * (elec_actual - elec_modelled) / elec_actual, 100000])[1]
            max_grid_dist = \
            sorted([0.5, max_grid_dist + max_grid_dist * 2 * (elec_actual - elec_modelled) / elec_actual, 60])[1]
            max_road_dist = \
            sorted([0.5, max_road_dist + max_road_dist * 2 * (elec_actual - elec_modelled) / elec_actual, 5])[1]
        elif elec_modelled - elec_actual < 0:
            pop_cutoff2 = sorted([0.01, pop_cutoff2 - pop_cutoff2 *
                                  (elec_actual - elec_modelled) / elec_actual, 100000])[1]
        elif elec_modelled - elec_actual > 0:
            pop_cutoff = sorted([0.01, pop_cutoff - pop_cutoff * 0.5 *
                                 (elec_actual - elec_modelled) / elec_actual, 10000])[1]
            #
        constraints = '{}{}{}{}{}'.format(pop_cutoff, min_night_lights, max_grid_dist, max_road_dist, pop_cutoff2)
        if constraints in prev_vals and not is_round_two:
            logging.info('Repeating myself, on to round two')
            prev_vals = []
            is_round_two = True
        elif constraints in prev_vals and is_round_two:
            logging.info('NOT SATISFIED: repeating myself')
            print('2. Modelled electrification rate = {}'.format(elec_modelled))
            if 'y' in input('Do you want to rerun calibration with new input values? <y/n>'):
                count = 0
                is_round_two = False
                pop_cutoff = float(input('Enter value for pop_cutoff: '))
                min_night_lights = float(input('Enter value for min_night_lights: '))
                max_grid_dist = float(input('Enter value for max_grid_dist: '))
                max_road_dist = float(input('Enter value for max_road_dist: '))
                pop_cutoff2 = float(input('Enter value for pop_cutoff2: '))
                break
        else:
            prev_vals.append(constraints)

        if count >= max_iterations_one and not is_round_two:
            logging.info('Got to {}, on to round two'.format(max_iterations_one))
            is_round_two = True
        elif count >= max_iterations_two and is_round_two:
            logging.info('NOT SATISFIED: Got to {}'.format(max_iterations_two))
            print('2. Modelled electrification rate = {}'.format(elec_modelled))
            if 'y' in input('Do you want to rerun calibration with new input values? <y/n>'):
                count = 0
                is_round_two = False
                pop_cutoff = int(input('Enter value for pop_cutoff: '))
                min_night_lights = int(input('Enter value for min_night_lights: '))
                max_grid_dist = int(input('Enter value for max_grid_dist: '))
                max_road_dist = int(input('Enter value for max_road_dist: '))
                pop_cutoff2 = int(input('Enter value for pop_cutoff2: '))
            else:
                break

        count += 1
        rural_elec_ratio = 1
        urban_elec_ratio = 1

    logging.info('The modelled electrification rate achieved is {}. '
                 'If this is not acceptable please revise this part of the algorithm'.format(elec_modelled))
    condition = 1

    self.df[SET_ELEC_FUTURE_GRID + "{}".format(start_year)] = \
        self.df.apply(lambda row: 1 if row[SET_ELEC_CURRENT] == 1 else 0, axis=1)
    self.df[SET_ELEC_FUTURE_OFFGRID + "{}".format(start_year)] = self.df.apply(lambda row: 0, axis=1)
    self.df[SET_ELEC_FUTURE_ACTUAL + "{}".format(start_year)] = \
        self.df.apply(lambda row: 1 if row[SET_ELEC_FUTURE_GRID + "{}".format(start_year)] == 1 or
                                       row[SET_ELEC_FUTURE_OFFGRID + "{}".format(start_year)] == 1 else 0, axis=1)
    self.df[SET_ELEC_FINAL_CODE + "{}".format(start_year)] = \
        self.df.apply(lambda row: 1 if row[SET_ELEC_CURRENT] == 1 else 99, axis=1)

    return min_night_lights, dist_to_trans, max_grid_dist, max_road_dist, elec_modelled, pop_cutoff, pop_cutoff2, rural_elec_ratio, urban_elec_ratio


if __name__ == "__main__":
    GIS_files_path, specs_path = sys.argv[1:]
    main(GIS_files_path, specs_path)

## 6. Create electrified and un-electrified demandhubs

- In GIS join the step 3. gridded shape file with step 5 shape file.

## 7. Assign demand to demandhubs

In GIS do:
- raster to point with GDP to demand hubs.
calculate the electrified population
with following equation
![alt text](Isolated.png "Title")
